In [1]:
from PIL import Image, ImageOps
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import image
import glob, os
import math
from scipy import spatial
import random 

In [ ]:
def load_images(source:str) -> np.ndarray:
    with Image.open(source) as im:
        im_arr = np.asarray(im)
    return im_arr

In [ ]:
face_im_arr = load_images('images/SampleImage.webp')
plt.imshow(Image.fromarray(face_im_arr))


In [ ]:
width = face_im_arr.shape[0]
height = face_im_arr.shape[1]

target_res = (20, 20) #number of rows and column

In [ ]:
face_im_arr.shape

In [ ]:
mos_template = face_im_arr[::(height//target_res[0]),::(height//target_res[1])]

In [ ]:
plt.imshow(Image.fromarray(mos_template[0,0]))

In [ ]:
plt.imshow(Image.fromarray(mos_template[::0, ::0]))

In [ ]:
mos_template[:,:, -1].size

In [ ]:
images = []

for file in glob.glob('Mosaic-Images/*'):
    im = load_images(file)
    images.append(im)

In [ ]:
images = [i for i in images if i.ndim==3]

In [ ]:
def resize_image(img : Image, size : tuple) -> np.ndarray:
    resz_img = ImageOps.fit(img, size, Image.LANCZOS, centering=(0.5, 0.5))
    return np.array(resz_img)

In [ ]:
mosaic_size = (40, 40) ## Defines size of each mosiac image
images = [resize_image(Image.fromarray(i), mosaic_size) for i in images]

In [ ]:
len(images)

In [ ]:
plt.imshow(Image.fromarray(images[60]))

In [ ]:
images_array = np.asarray(images)
images_array.shape

In [ ]:
image_values = np.apply_over_axes(np.mean, images_array, [1,2]).reshape(len(images),3)
image_values

In [ ]:
tree = spatial.KDTree(image_values)

In [ ]:
image_idx = np.zeros(target_res, dtype=np.uint32)


for i in range(target_res[0]):
    for j in range(target_res[1]):
        
        template = mos_template[i, j]
        
        match = tree.query(template, k=40)
        pick = random.randint(0, 39)
        image_idx[i, j] = match[1][pick]

In [ ]:
canvas = Image.new('RGB', (mosaic_size[0]*target_res[0], mosaic_size[1]*target_res[1]))

for i in range(target_res[0]):
    for j in range(target_res[1]):
        arr = images[image_idx[j, i]]
        x, y = i*mosaic_size[0], j*mosaic_size[1]
        im = Image.fromarray(arr)
        canvas.paste(im, (x,y))
        
canvas

In [ ]:
canvas = Image.new('RGB', (mosaic_size[0]*target_res[0], mosaic_size[1]*target_res[1]))

In [ ]:
plt.imshow(Image.fromarray(image_idx[19]))

In [ ]:
!pip3 install BeautifulSoup4

In [ ]:
import os
import json 
import requests 
from bs4 import BeautifulSoup 

In [ ]:
import os
import json 
import requests 
from bs4 import BeautifulSoup 

GOOGLE_IMAGE = \
    'https://www.google.com/search?site=&tbm=isch&source=hp&biw=1873&bih=990&'

# The User-Agent request header contains a characteristic string 
# that allows the network protocol peers to identify the application type, 
# operating system, and software version of the requesting software user agent.
# needed for google search
usr_agent = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
    'Accept-Encoding': 'none',
    'Accept-Language': 'en-US,en;q=0.8',
    'Connection': 'keep-alive',
}

i=0
SAVE_FOLDER = 'OtherImages'
filename_tosave = "box"
dir = SAVE_FOLDER+'/'


# def main():
#     if not os.path.exists(SAVE_FOLDER):
#         print("Making Directory")
#         os.mkdir(SAVE_FOLDER)
#     print("Directory exists")
#     download_images()
    
def download_images(keyword):
    
    searchurl = GOOGLE_IMAGE + 'q=' + keyword
    print(searchurl)

    # request url, without usr_agent the permission gets denied
    response = requests.get(searchurl, headers=usr_agent)
    html = response.text
    
    #Parsing HTML
    soup = BeautifulSoup(html, 'html.parser')
    
    #Retrieveing number of files present in the folder
    global i
    imagelinks= []
    list = os.listdir(dir) # dir is your directory path
    i = len(list)
    print(i)
    
    #Search for Class "img" and eventually obtain links in "data-src" 
    for link in soup.findAll('img'):
        op = link.get('data-src')
        if not op:
            continue
        else:
            response = requests.get(op)
            imagename = SAVE_FOLDER + '/' + filename_tosave + str(i+1) + '.jpg'
            with open(imagename, 'wb') as file:
                file.write(response.content)
            i = i +1
            print("IMAGE LINKS:", link.get('data-src'))

    print('Done Downloading')

In [ ]:
download_images('food')

In [ ]:
tile_images = [load_images(file) for file in glob.glob('Mosaic_Images'+'/*')]


In [ ]:
tile_images = []

for file in glob.glob('Mosaic-Images'+'/*'):
    im = load_images(file)
    tile_images.append(im)

In [ ]:
len(tile_images)

In [ ]:
tile_images = [i for i in tile_images if i.ndim==3]
tile_images = [resize_image(Image.fromarray(i), (40,40)) for i in tile_images]
tile_images_array = np.asarray(tile_images)
tile_images_values = np.apply_over_axes(np.mean, tile_images_array, [1,2]).reshape(len(tile_images),3)

In [ ]:
from photomosaic import Photomosaic


result = Photomosaic('Mosaic-Images', 'images/SampleImage.webp','output', mosaic_size=100, divisions=40)
result.process()

In [2]:
import glob

files = []
for file in glob.glob('Mosaic-Images'+'/*'):
    files.append(file)

files

['Mosaic-Images/47912.png',
 'Mosaic-Images/47906.png',
 'Mosaic-Images/47090.png',
 'Mosaic-Images/47084.png',
 'Mosaic-Images/47709.png',
 'Mosaic-Images/47053.png',
 'Mosaic-Images/47735.png',
 'Mosaic-Images/47721.png',
 'Mosaic-Images/47047.png',
 'Mosaic-Images/47292.png',
 'Mosaic-Images/47286.png',
 'Mosaic-Images/47279.png',
 'Mosaic-Images/47537.png',
 'Mosaic-Images/47251.png',
 'Mosaic-Images/47245.png',
 'Mosaic-Images/47523.png',
 'Mosaic-Images/47480.png',
 'Mosaic-Images/47494.png',
 'Mosaic-Images/47319.png',
 'Mosaic-Images/47443.png',
 'Mosaic-Images/47325.png',
 'Mosaic-Images/47331.png',
 'Mosaic-Images/47457.png',
 'Mosaic-Images/47866.png',
 'Mosaic-Images/47872.png',
 'Mosaic-Images/47682.png',
 'Mosaic-Images/47696.png',
 'Mosaic-Images/47669.png',
 'Mosaic-Images/47127.png',
 'Mosaic-Images/47641.png',
 'Mosaic-Images/47899.png',
 'Mosaic-Images/47655.png',
 'Mosaic-Images/47133.png',
 'Mosaic-Images/47654.png',
 'Mosaic-Images/47132.png',
 'Mosaic-Images/4712

In [3]:
files = files[:10]

In [4]:
files

['Mosaic-Images/47912.png',
 'Mosaic-Images/47906.png',
 'Mosaic-Images/47090.png',
 'Mosaic-Images/47084.png',
 'Mosaic-Images/47709.png',
 'Mosaic-Images/47053.png',
 'Mosaic-Images/47735.png',
 'Mosaic-Images/47721.png',
 'Mosaic-Images/47047.png',
 'Mosaic-Images/47292.png']

In [1]:
from photomosaic import Photomosaic
import glob

files = []
for file in glob.glob('Mosaic-Images'+'/*'):
    files.append(file)

files = files[:10]
result = Photomosaic(files, 'images/1.jpeg','output', mosaic_size=100, divisions=40,tile_choice=5)
result.process()

Main Image process completed: 226 by 223
10
Tile Image process completed: 10 tile images
Matching tiles to sliding windows: 10 tile images
array (100, 100, 3)
template  (3,)
resized templated (100, 100, 3)
array (100, 100, 3)
template  (3,)
resized templated (100, 100, 3)
array (100, 100, 3)
template  (3,)
resized templated (100, 100, 3)
array (100, 100, 3)
template  (3,)
resized templated (100, 100, 3)
array (100, 100, 3)
template  (3,)
resized templated (100, 100, 3)
array (100, 100, 3)
template  (3,)
resized templated (100, 100, 3)
array (100, 100, 3)
template  (3,)
resized templated (100, 100, 3)
array (100, 100, 3)
template  (3,)
resized templated (100, 100, 3)
array (100, 100, 3)
template  (3,)
resized templated (100, 100, 3)
array (100, 100, 3)
template  (3,)
resized templated (100, 100, 3)
array (100, 100, 3)
template  (3,)
resized templated (100, 100, 3)
array (100, 100, 3)
template  (3,)
resized templated (100, 100, 3)
array (100, 100, 3)
template  (3,)
resized templated (100

'output/Bf7YN_mosaic.jpg'

In [ ]:
import numpy as np
from PIL import Image

# Converting image to numpy array
# data = Image.open("wall.jpg")

# white = np.array([255, 255, 255], np.uint8)
# vector = white-data

# percent = 0.5
# value = data + vector * percent

# img = Image.fromarray(value.astype(np.uint8), 'RGB')
# img.save('result.png')

source = "images/1.jpeg"
with Image.open(source) as im:
    white = np.array([255, 255, 255], np.uint8)
    vector = white-im

    percent = 0.5
    value = im + vector * percent

    img = Image.fromarray(value.astype(np.uint8), 'RGB')
    img.show()